# A2
* GCR = 0.4
* Albedo = 0.5
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A2-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A2-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+249.g0e452df.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A2-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.5
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A2-1
Loading albedo, 1 value(s), 0.500 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-11-01_0000', endtime='2022-12-01_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~281 skyfiles. 
Created 263 skyfiles in /skies/


{'2022-11-01_0630': {'surf_azm': 90.0,
  'surf_tilt': 13.77,
  'theta': -13.77,
  'dni': 497,
  'ghi': 64,
  'dhi': 17,
  'temp_air': 6.0,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0630.rad'},
 '2022-11-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 35.73,
  'theta': -35.73,
  'dni': 721,
  'ghi': 174,
  'dhi': 27,
  'temp_air': 7.4,
  'wind_speed': 0.3,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0730.rad'},
 '2022-11-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 898,
  'ghi': 372,
  'dhi': 37,
  'temp_air': 9.9,
  'wind_speed': 0.8,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0830.rad'},
 '2022-11-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 38.6,
  'theta': -38.6,
  'dni': 967,
  'ghi': 541,
  'dhi': 47,
  'temp_air': 12.6,
  'wind_speed': 2.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0930.rad'},
 '2022-11-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 18.33,
  'theta': -18.33,
  'dni': 870,
  'ghi': 636,
  'd

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~263 .rad files for gendaylit 1-axis workflow (this takes a minute..)
263 Radfiles created in /objects/

Making 263 octfiles in root directory.
Created 1axis_2022-11-01_0630.oct
Created 1axis_2022-11-01_0730.oct
Created 1axis_2022-11-01_0830.oct
Created 1axis_2022-11-01_0930.oct
Created 1axis_2022-11-01_1030.oct
Created 1axis_2022-11-01_1130.oct
Created 1axis_2022-11-01_1230.oct
Created 1axis_2022-11-01_1330.oct
Created 1axis_2022-11-01_1430.oct
Created 1axis_2022-11-01_1530.oct
Created 1axis_2022-11-02_0730.oct
Created 1axis_2022-11-02_0830.oct
Created 1axis_2022-11-02_0930.oct
Created 1axis_2022-11-02_1030.oct
Created 1axis_2022-11-02_1130.oct
Created 1axis_2022-11-02_1230.oct
Created 1axis_2022-11-02_1330.oct
Created 1axis_2022-11-02_1430.oct
Created 1axis_2022-11-02_1530.oct
Created 1axis_2022-11-02_1630.oct
Created 1axis_2022-11-03_0630.oct
Created 1axis_2022-11-03_0730.oct
Created 1axis_2022-11-03_0830.oct
Created 1axis_2022-11-03_0930.oct
Created 1axis_2022-11-03_1030.oc

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.172789), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-04_1630.oct
Created 1axis_2022-11-05_0730.oct
Created 1axis_2022-11-05_0830.oct
Created 1axis_2022-11-05_0930.oct
Created 1axis_2022-11-05_1030.oct
Created 1axis_2022-11-05_1130.oct
Created 1axis_2022-11-05_1230.oct
Created 1axis_2022-11-05_1330.oct
Created 1axis_2022-11-05_1430.oct
Created 1axis_2022-11-05_1530.oct
Created 1axis_2022-11-05_1630.oct
Created 1axis_2022-11-06_0630.oct
Created 1axis_2022-11-06_0730.oct
Created 1axis_2022-11-06_0830.oct
Created 1axis_2022-11-06_0930.oct
Created 1axis_2022-11-06_1030.oct
Created 1axis_2022-11-06_1130.oct
Created 1axis_2022-11-06_1230.oct
Created 1axis_2022-11-06_1330.oct
Created 1axis_2022-11-06_1430.oct
Created 1axis_2022-11-06_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.246035), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-06_1630.oct
Created 1axis_2022-11-07_0730.oct
Created 1axis_2022-11-07_0830.oct
Created 1axis_2022-11-07_0930.oct
Created 1axis_2022-11-07_1030.oct
Created 1axis_2022-11-07_1130.oct
Created 1axis_2022-11-07_1230.oct
Created 1axis_2022-11-07_1330.oct
Created 1axis_2022-11-07_1430.oct
Created 1axis_2022-11-07_1530.oct
Created 1axis_2022-11-07_1630.oct
Created 1axis_2022-11-08_0730.oct
Created 1axis_2022-11-08_0830.oct
Created 1axis_2022-11-08_0930.oct
Created 1axis_2022-11-08_1030.oct
Created 1axis_2022-11-08_1130.oct
Created 1axis_2022-11-08_1230.oct
Created 1axis_2022-11-08_1330.oct
Created 1axis_2022-11-08_1430.oct
Created 1axis_2022-11-08_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.139933), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-08_1630.oct
Created 1axis_2022-11-09_0730.oct
Created 1axis_2022-11-09_0830.oct
Created 1axis_2022-11-09_0930.oct
Created 1axis_2022-11-09_1030.oct
Created 1axis_2022-11-09_1130.oct
Created 1axis_2022-11-09_1230.oct
Created 1axis_2022-11-09_1330.oct
Created 1axis_2022-11-09_1430.oct
Created 1axis_2022-11-09_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.254273), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-09_1630.oct
Created 1axis_2022-11-10_0630.oct
Created 1axis_2022-11-10_0730.oct
Created 1axis_2022-11-10_0830.oct
Created 1axis_2022-11-10_0930.oct
Created 1axis_2022-11-10_1030.oct
Created 1axis_2022-11-10_1130.oct
Created 1axis_2022-11-10_1230.oct
Created 1axis_2022-11-10_1330.oct
Created 1axis_2022-11-10_1430.oct
Created 1axis_2022-11-10_1530.oct
Created 1axis_2022-11-11_0730.oct
Created 1axis_2022-11-11_0830.oct
Created 1axis_2022-11-11_0930.oct
Created 1axis_2022-11-11_1030.oct
Created 1axis_2022-11-11_1130.oct
Created 1axis_2022-11-11_1230.oct
Created 1axis_2022-11-11_1330.oct
Created 1axis_2022-11-11_1430.oct
Created 1axis_2022-11-11_1530.oct
Created 1axis_2022-11-12_0730.oct
Created 1axis_2022-11-12_0830.oct
Created 1axis_2022-11-12_0930.oct
Created 1axis_2022-11-12_1030.oct
Created 1axis_2022-11-12_1130.oct
Created 1axis_2022-11-12_1230.oct
Created 1axis_2022-11-12_1330.oct
Created 1axis_2022-11-12_1430.oct
Created 1axis_2022-11-12_1530.oct
Created 1axis_

In [ ]:
trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

Linescan in process: 1axis_2022-11-01_0630_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0630_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0630_Row3_Module13.csv
Index: 2022-11-01_0630. Wm2Front: 193.69744999999998. Wm2Back: 4.586875666666667
Linescan in process: 1axis_2022-11-01_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0730_Row3_Module13.csv
Index: 2022-11-01_0730. Wm2Front: 541.2474166666667. Wm2Back: 8.284640833333334
Linescan in process: 1axis_2022-11-01_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0830_Row3_Module13.csv
Index: 2022-11-01_0830. Wm2Front: 752.73135. Wm2Back: 47.39756166666667
Linescan in process: 1axis_2022-11-01_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0930_Row3_Module13.csv
Index: 2022-11-01_0930. Wm2Fron

Linescan in process: 1axis_2022-11-04_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_0930_Row3_Module13.csv
Index: 2022-11-04_0930. Wm2Front: 358.0702. Wm2Back: 58.98202833333333
Linescan in process: 1axis_2022-11-04_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1030_Row3_Module13.csv
Index: 2022-11-04_1030. Wm2Front: 76.80708166666666. Wm2Back: 20.60053
Linescan in process: 1axis_2022-11-04_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1130_Row3_Module13.csv
Index: 2022-11-04_1130. Wm2Front: 104.26583333333333. Wm2Back: 26.408406666666664
Linescan in process: 1axis_2022-11-04_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1230_Row3_Module13.csv
Index: 2022-11-04_1230. Wm2Front: 180.16021666666666. Wm2Back: 48.48990666666666
Linescan in process: 1

Linescan in process: 1axis_2022-11-07_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_0930_Row3_Module13.csv
Index: 2022-11-07_0930. Wm2Front: 726.1963. Wm2Back: 79.19370833333333
Linescan in process: 1axis_2022-11-07_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1030_Row3_Module13.csv
Index: 2022-11-07_1030. Wm2Front: 690.0350833333333. Wm2Back: 122.57256666666666
Linescan in process: 1axis_2022-11-07_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1130_Row3_Module13.csv
Index: 2022-11-07_1130. Wm2Front: 566.4748333333332. Wm2Back: 117.26050000000001
Linescan in process: 1axis_2022-11-07_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1230_Row3_Module13.csv
Index: 2022-11-07_1230. Wm2Front: 692.0378333333333. Wm2Back: 113.66314999999999
Linescan in p

Linescan in process: 1axis_2022-11-10_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_0930_Row3_Module13.csv
Index: 2022-11-10_0930. Wm2Front: 719.6976833333333. Wm2Back: 72.83214666666666
Linescan in process: 1axis_2022-11-10_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1030_Row3_Module13.csv
Index: 2022-11-10_1030. Wm2Front: 672.9838500000001. Wm2Back: 113.13718333333334
Linescan in process: 1axis_2022-11-10_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1130_Row3_Module13.csv
Index: 2022-11-10_1130. Wm2Front: 685.3688833333333. Wm2Back: 132.0875
Linescan in process: 1axis_2022-11-10_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1230_Row3_Module13.csv
Index: 2022-11-10_1230. Wm2Front: 716.45985. Wm2Back: 106.37953166666668
Linescan in process: 1

In [ ]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

In [ ]:
#trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [ ]:
# demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A2-Dec.csv')

In [ ]:
# demo.exportTrackerDict(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A2-Dec-27-M2.csv')

In [ ]:
# trackerdict

In [ ]:
demo.CompiledResults.to_csv(Path().resolve().parent.parent / 'Results' / 'A2-all-Nov.csv')